In [1]:
import pandas as pd
import numpy as np

from pandas import NA

import re

# Loading data

In [2]:
house = pd.read_csv('[nov 20] lamudi_house.csv')
apt = pd.read_csv('[nov 20] lamudi_apt.csv')
condo = pd.read_csv('[nov 20] lamudi_condo.csv')
comm = pd.read_csv('[nov 20] lamudi_commercial.csv')


# Exploratory Data Analysis

- Data Cleaning
    - NA Values
        - Check for keywords in description to fill NA values
        - Drop columns that are mostly NA
        - Drop rows that have NA values
    - Duplicate Values
        - Drop duplicate values

## Data Cleaning

### NA and Duplicate Values

In [3]:
house = house.replace({pd.NA: NA, np.nan: NA, 'NaN': NA})
apt = apt.replace({pd.NA: NA, np.nan: NA, 'NaN': NA})
condo = condo.replace({pd.NA: NA, np.nan: NA, 'NaN': NA})
comm = comm.replace({pd.NA: NA, np.nan: NA, 'NaN': NA})

In [4]:
# List of source DataFrames
dataframes = [house, condo, apt, comm]
df_names = ["house", "condominium", "apartment", "commercial"]

# Collect NA counts in each DataFrame
na_counts_data = []
for df, name in zip(dataframes, df_names):
    # Replace "na" text with NaN if needed
    df.replace("na", pd.NA, inplace=True)
    
    # Count NA values in each column
    na_counts = df.isna().sum()
    
    # Append results with the source DataFrame name
    for column, count in na_counts.items():
        na_counts_data.append([name, column, count])

# Create the summary DataFrame
na_counts_df = pd.DataFrame(na_counts_data, columns=["Listing Type", "Feature", "NA Count"])

na_counts_df[na_counts_df["Listing Type"] == "commercial"]

,Listing Type,Feature,NA Count
47,commercial,title,1
48,commercial,price,0
49,commercial,location,0
50,commercial,description,0
51,commercial,bedrooms,1454
52,commercial,bathrooms,1452
53,commercial,floor_area,184
54,commercial,property_type,0
55,commercial,subcategories,0
56,commercial,listing_url,0


In [5]:
def clean_house_description(df, inplace=True):
    """
    Clean housing DataFrame by extracting features from description text.
    
    Args:
        df (pandas.DataFrame): DataFrame containing housing data with 'description' column
        inplace (bool): If True, modifies the original DataFrame; if False, returns a copy
    
    Returns:
        pandas.DataFrame: Original DataFrame with extracted features if inplace=True, 
                        or a new DataFrame if inplace=False
    """
    def safe_float_convert(value):
        """Safely convert string to float, returning None if conversion fails"""
        try:
            if pd.isna(value) or value == '':
                return None
            if isinstance(value, str):
                # Remove commas and spaces
                value = value.replace(',', '').strip()
            return float(value)
        except (ValueError, TypeError):
            return None

    def extract_features(description):
        if not isinstance(description, str):
            return {}
        
        features = {}
        
        # Extract bedrooms
        bedroom_pattern = r'(\d+)\s*(?:Bedroom|bedroom|BEDROOM)'
        bedroom_match = re.search(bedroom_pattern, description)
        if bedroom_match:
            try:
                features['bedrooms'] = int(bedroom_match.group(1))
            except ValueError:
                pass
            
        # Extract bathrooms (toilet and bath)
        bathroom_pattern = r'(\d+)\s*(?:Toilet and Bath|toilet and bath|T&B|bathroom|BATHROOM)'
        bathroom_match = re.search(bathroom_pattern, description)
        if bathroom_match:
            try:
                features['bathrooms'] = int(bathroom_match.group(1))
            except ValueError:
                pass
            
        # Extract car spaces/garage
        car_pattern = r'(\d+)\s*(?:Car Garage|car garage|parking space|CAR GARAGE|PARKING SPACE)'
        car_match = re.search(car_pattern, description)
        if car_match:
            try:
                features['car_spaces'] = int(car_match.group(1))
            except ValueError:
                pass
    
        
        return features
    
    # Work with original DataFrame if inplace=True, otherwise create a copy
    cleaned_df = df if inplace else df.copy()
    
    # Convert any empty strings to NaN in numeric columns
    numeric_columns = ['bedrooms', 'bathrooms', 'car_spaces']
    for col in numeric_columns:
        if col in cleaned_df.columns:
            cleaned_df[col] = cleaned_df[col].apply(safe_float_convert)
    
    # Extract features from each description
    extracted_features = cleaned_df['description'].apply(extract_features)
    
    # Update DataFrame with extracted features
    update_count = 0
    for row_idx, features in extracted_features.items():
        for feature, value in features.items():
            if feature in cleaned_df.columns and pd.isna(cleaned_df.at[row_idx, feature]):
                cleaned_df.at[row_idx, feature] = value
                update_count += 1
    
    print(f"Updated {update_count} empty fields with extracted data")
    return cleaned_df

def process_housing_data(df, inplace=True):
    """
    Process housing data
    
    Args:
        df (pandas.DataFrame): Input DataFrame with housing data
        inplace (bool): If True, modifies the original DataFrame
    
    Returns:
        pandas.DataFrame: Processed DataFrame
    """
    # Clean the data
    cleaned_df = clean_house_description(df, inplace=inplace)
    
    # Print summary of the cleaning process
    total_rows = len(cleaned_df)
    columns_to_check = ['bedrooms', 'bathrooms', 'car_spaces', 'floor_area']
    filled_counts = {}
    
    for col in columns_to_check:
        if col in cleaned_df.columns:
            filled_counts[col] = cleaned_df[col].notna().sum()
    
    print("\nData Cleaning Summary:")
    print(f"Total number of records: {total_rows}")
    print("\nFilled values for each feature:")
    for feature, count in filled_counts.items():
        percentage = (count / total_rows) * 100
        print(f"{feature}: {count} ({percentage:.1f}%)")
    
    return cleaned_df

# Example usage
if __name__ == "__main__":
    try:
        # Process the data in-place (modifies original DataFrame)
        process_housing_data(house, inplace=True)
        
    except Exception as e:
        print(f"Error processing data: {str(e)}")
        raise

Updated 656 empty fields with extracted data

Data Cleaning Summary:
Total number of records: 3000

Filled values for each feature:
bedrooms: 2948 (98.3%)
bathrooms: 2833 (94.4%)
car_spaces: 2038 (67.9%)
floor_area: 2890 (96.3%)


In [6]:
def clean_apt_description(df, inplace=True):
    def safe_float_convert(value):
        """Safely convert string to float, returning None if conversion fails"""
        try:
            if pd.isna(value) or value == '':
                return None
            if isinstance(value, str):
                # Remove commas and common suffixes
                value = value.replace(',', '').strip()
                value = value.lower()
                # Convert K/M suffix to actual numbers
                if value.endswith('k'):
                    value = float(value[:-1]) * 1000
                elif value.endswith('m'):
                    value = float(value[:-1]) * 1000000
                # Remove common area suffixes
                value = re.sub(r'sq\.?m?$', '', value)
            return float(value)
        except (ValueError, TypeError):
            return None

    def extract_features(description):
        if not isinstance(description, str):
            return {}
        
        features = {}
        
        # Pattern for both colon-separated and direct number formats
        patterns = {
            'bedrooms': [
                r'(?:Bedroom|bedroom|BEDROOM)[s\s]*[:]\s*(\d+)',
                r'(\d+)\s*(?:Bedroom|bedroom|BEDROOM)',
            ],
            'bathrooms': [
                r'(?:Toilet and Bath|toilet and bath|T&B|bathroom|BATHROOM)[s\s]*[:]\s*(\d+)',
                r'(\d+)\s*(?:Toilet and Bath|toilet and bath|T&B|bathroom|BATHROOM)',
            ],
            'car_spaces': [
                r'(?:Car Garage|car garage|parking space|CAR GARAGE|PARKING SPACE)[s\s]*[:]\s*(\d+)',
                r'(\d+)\s*(?:Car Garage|car garage|parking space|CAR GARAGE|PARKING SPACE)',
            ],
            'floor_area': [
                r'(?:Floor area|floor area|Floor Area|FLOOR AREA)[s\s]*[:]\s*(\d+(?:,\d+)*(?:\.\d+)?)\s*(?:sq\.?m?)?',
                r'(\d+(?:,\d+)*(?:\.\d+)?)\s*(?:sq\.?m?)?\s*(?:Floor area|floor area|Floor Area|FLOOR AREA)',
            ],
            'land_size': [
                r'(?:Lot area|lot area|Lot Area|LOT AREA)[s\s]*[:]\s*(\d+(?:,\d+)*(?:\.\d+)?)\s*(?:sq\.?m?)?',
                r'(\d+(?:,\d+)*(?:\.\d+)?)\s*(?:sq\.?m?)?\s*(?:Lot area|lot area|Lot Area|LOT AREA)',
            ]
        }
        
        # Extract features using patterns
        for feature, pattern_list in patterns.items():
            for pattern in pattern_list:
                match = re.search(pattern, description)
                if match:
                    value = match.group(1)
                    # Convert the value to float
                    converted_value = safe_float_convert(value)
                    if converted_value is not None:
                        features[feature] = converted_value
                        break  # Stop checking patterns for this feature once found
                        
        
            
        return features
    
    # Work with original DataFrame if inplace=True, otherwise create a copy
    cleaned_df = df if inplace else df.copy()
    
    # Convert any empty strings to NaN in numeric columns
    numeric_columns = ['bedrooms', 'bathrooms', 'car_spaces', 'floor_area', 'land_size']
    for col in numeric_columns:
        if col in cleaned_df.columns:
            cleaned_df[col] = cleaned_df[col].apply(safe_float_convert)
    
    # Extract features from each description
    extracted_features = cleaned_df['description'].apply(extract_features)
    
    # Update DataFrame with extracted features
    update_count = 0
    for row_idx, features in extracted_features.items():
        for feature, value in features.items():
            # Create new column if it doesn't exist
            if feature not in cleaned_df.columns:
                cleaned_df[feature] = None
            # Update value if current value is NaN or None
            if pd.isna(cleaned_df.at[row_idx, feature]):
                cleaned_df.at[row_idx, feature] = value
                update_count += 1
    
    print(f"Updated {update_count} empty fields with extracted data")
    return cleaned_df

def process_apt_data(df, inplace=True):
    # Clean the data
    cleaned_df = clean_apt_description(df, inplace=inplace)
    
    # Print summary of the cleaning process
    total_rows = len(cleaned_df)
    columns_to_check = ['bedrooms', 'bathrooms', 'car_spaces', 'floor_area', 'land_size']
    filled_counts = {}
    
    for col in columns_to_check:
        if col in cleaned_df.columns:
            filled_counts[col] = cleaned_df[col].notna().sum()
    
    print("\nData Cleaning Summary:")
    print(f"Total number of records: {total_rows}")
    print("\nFilled values for each feature:")
    for feature, count in filled_counts.items():
        percentage = (count / total_rows) * 100
        print(f"{feature}: {count} ({percentage:.1f}%)")
    
    return cleaned_df

# Example usage
if __name__ == "__main__":
    try:
        # Process the data in-place (modifies original DataFrame)
        process_apt_data(apt, inplace=True)
        
    except Exception as e:
        print(f"Error processing data: {str(e)}")
        raise

Updated 51 empty fields with extracted data

Data Cleaning Summary:
Total number of records: 72

Filled values for each feature:
bedrooms: 72 (100.0%)
bathrooms: 46 (63.9%)
car_spaces: 14 (19.4%)
floor_area: 62 (86.1%)
land_size: 43 (59.7%)


In [7]:
def clean_comm_description(df, inplace=True):
    def safe_float_convert(value):
        """Safely convert string to float, returning None if conversion fails"""
        try:
            if pd.isna(value) or value == '':
                return None
            if isinstance(value, str):
                # Remove commas and common suffixes
                value = value.replace(',', '').strip()
                value = value.lower()
                # Convert K/M suffix to actual numbers
                if value.endswith('k'):
                    value = float(value[:-1]) * 1000
                elif value.endswith('m'):
                    value = float(value[:-1]) * 1000000
                # Remove common area suffixes
                value = re.sub(r'sq\.?m?$', '', value)
            return float(value)
        except (ValueError, TypeError):
            return None

    def extract_features(description):
        if not isinstance(description, str):
            return {}
        
        features = {}
        
        # Pattern for both colon-separated and direct number formats
        patterns = {
            'bedrooms': [
                r'(?:Bedroom|bedroom|BEDROOM)[s\s]*[:]\s*(\d+)',
                r'(\d+)\s*(?:Bedroom|bedroom|BEDROOM)',
            ],
            'bathrooms': [
                r'(?:Toilet and Bath|toilet and bath|T&B|bathroom|BATHROOM)[s\s]*[:]\s*(\d+)',
                r'(\d+)\s*(?:Toilet and Bath|toilet and bath|T&B|bathroom|BATHROOM)',
            ],
            'car_spaces': [
                r'(?:Car Garage|car garage|parking space|CAR GARAGE|PARKING SPACE)[s\s]*[:]\s*(\d+)',
                r'(\d+)\s*(?:Car Garage|car garage|parking space|CAR GARAGE|PARKING SPACE)',
            ],
            'floor_area': [
                r'(?:Floor area|floor area|Floor Area|FLOOR AREA)[s\s]*[:]\s*(\d+(?:,\d+)*(?:\.\d+)?)\s*(?:sq\.?m?)?',
                r'(\d+(?:,\d+)*(?:\.\d+)?)\s*(?:sq\.?m?)?\s*(?:Floor area|floor area|Floor Area|FLOOR AREA)',
            ],
            'land_size': [
                r'(?:Lot area|lot area|Lot Area|LOT AREA)[s\s]*[:]\s*(\d+(?:,\d+)*(?:\.\d+)?)\s*(?:sq\.?m?)?',
                r'(\d+(?:,\d+)*(?:\.\d+)?)\s*(?:sq\.?m?)?\s*(?:Lot area|lot area|Lot Area|LOT AREA)',
            ]
        }
        
        # Extract features using patterns
        for feature, pattern_list in patterns.items():
            for pattern in pattern_list:
                match = re.search(pattern, description)
                if match:
                    value = match.group(1)
                    # Convert the value to float
                    converted_value = safe_float_convert(value)
                    if converted_value is not None:
                        features[feature] = converted_value
                        break  # Stop checking patterns for this feature once found
                        
        
            
        return features
    
    # Work with original DataFrame if inplace=True, otherwise create a copy
    cleaned_df = df if inplace else df.copy()
    
    # Convert any empty strings to NaN in numeric columns
    numeric_columns = ['bedrooms', 'bathrooms', 'car_spaces', 'floor_area', 'land_size']
    for col in numeric_columns:
        if col in cleaned_df.columns:
            cleaned_df[col] = cleaned_df[col].apply(safe_float_convert)
    
    # Extract features from each description
    extracted_features = cleaned_df['description'].apply(extract_features)
    
    # Update DataFrame with extracted features
    update_count = 0
    for row_idx, features in extracted_features.items():
        for feature, value in features.items():
            # Create new column if it doesn't exist
            if feature not in cleaned_df.columns:
                cleaned_df[feature] = None
            # Update value if current value is NaN or None
            if pd.isna(cleaned_df.at[row_idx, feature]):
                cleaned_df.at[row_idx, feature] = value
                update_count += 1
    
    print(f"Updated {update_count} empty fields with extracted data")
    return cleaned_df

def process_comm_data(df, inplace=True):
    # Clean the data
    cleaned_df = clean_comm_description(df, inplace=inplace)
    
    # Print summary of the cleaning process
    total_rows = len(cleaned_df)
    columns_to_check = ['bedrooms', 'bathrooms', 'car_spaces', 'floor_area', 'land_size']
    filled_counts = {}
    
    for col in columns_to_check:
        if col in cleaned_df.columns:
            filled_counts[col] = cleaned_df[col].notna().sum()
    
    print("\nData Cleaning Summary:")
    print(f"Total number of records: {total_rows}")
    print("\nFilled values for each feature:")
    for feature, count in filled_counts.items():
        percentage = (count / total_rows) * 100
        print(f"{feature}: {count} ({percentage:.1f}%)")
    
    return cleaned_df

# Example usage
if __name__ == "__main__":
    try:
        # Process the data in-place (modifies original DataFrame)
        process_apt_data(comm, inplace=True)
        
    except Exception as e:
        print(f"Error processing data: {str(e)}")
        raise

Updated 827 empty fields with extracted data

Data Cleaning Summary:
Total number of records: 1454

Filled values for each feature:
bedrooms: 30 (2.1%)
bathrooms: 11 (0.8%)
car_spaces: 55 (3.8%)
floor_area: 1273 (87.6%)
land_size: 730 (50.2%)


In [8]:
# List of source DataFrames
dataframes = [condo]
df_names = ["condominium"]

# Collect NA counts in each DataFrame
na_counts_data = []
for df, name in zip(dataframes, df_names):
    # Replace "na" text with NaN if needed
    df.replace("na", pd.NA, inplace=True)
    
    # Count NA values in each column
    na_counts = df.isna().sum()
    
    # Append results with the source DataFrame name
    for column, count in na_counts.items():
        na_counts_data.append([name, column, count])

# Create the summary DataFrame
na_counts_df = pd.DataFrame(na_counts_data, columns=["Listing Type", "Feature", "NA Count"])

na_counts_df

,Listing Type,Feature,NA Count
0,condominium,title,0
1,condominium,price,0
2,condominium,location,0
3,condominium,description,1
4,condominium,bedrooms,2
5,condominium,bathrooms,139
6,condominium,rooms_total,3000
7,condominium,floor_area,5
8,condominium,land_size,3000
9,condominium,listing_url,0


In [9]:
#house_dropNA = house.drop(columns = ["classification", "car_spaces", "subdivision_name", "property_type" ])
#house_dropNA = house_dropNA.drop_duplicates()
#
#
#apt_dropNA = apt.drop(columns = ["subcategories", "rooms_total", "car_spaces", "classification", "subdivision_name"])
#apt_dropNA = apt_dropNA.drop_duplicates()
#
#condo_dropNA = condo.drop(columns = ["rooms_total", "land_size", "car_spaces", "classification"])
#condo_dropNA = condo_dropNA.drop_duplicates()
#
#
#comm_dropNA = comm.drop(columns = ["bedrooms", "bathrooms", "property_type", "subcategories", "furnished_status",
#                                  "condominium_name", "land_size", "car_spaces"])
#comm_dropNA = comm_dropNA.drop_duplicates()

In [10]:
house_dropNA = house.drop(columns = ["classification", "car_spaces", "subdivision_name", "property_type" ])
house_dropNA = house_dropNA.dropna()
house_dropNA = house_dropNA.drop_duplicates()

condo_dropNA = condo.drop(columns = ["rooms_total", "land_size", "car_spaces", "classification"])
condo_dropNA = condo_dropNA.dropna()
condo_dropNA = condo_dropNA.drop_duplicates()

apt_dropNA = apt.drop(columns = ["subcategories", "rooms_total", "car_spaces", "classification", "subdivision_name"])
apt_dropNA = apt_dropNA.dropna()
apt_dropNA = apt_dropNA.drop_duplicates()

comm_dropNA = comm.drop(columns = ["bedrooms", "bathrooms", "property_type", "subcategories", "furnished_status",
                                  "condominium_name", "land_size", "car_spaces"])
comm_dropNA = comm_dropNA.dropna()
comm_dropNA = comm_dropNA.drop_duplicates()

In [11]:
print(f'Original DataFrame v Cleaned DataFrame')
print(f'{house.shape} v {house_dropNA.shape}')
print(f'{condo.shape} v {condo_dropNA.shape}')
print(f'{apt.shape} v {apt_dropNA.shape}')
print(f'{comm.shape} v {comm_dropNA.shape}')

Original DataFrame v Cleaned DataFrame
(3000, 16) v (1278, 12)
(3000, 15) v (1288, 11)
(72, 16) v (23, 11)
(1454, 17) v (529, 9)


In [12]:
house_dropNA = house_dropNA.replace({pd.NA: NA, np.nan: NA, 'NaN': NA})
apt_dropNA = apt_dropNA.replace({pd.NA: NA, np.nan: NA, 'NaN': NA})
condo_dropNA = condo_dropNA.replace({pd.NA: NA, np.nan: NA, 'NaN': NA})
comm_dropNA = comm_dropNA.replace({pd.NA: NA, np.nan: NA, 'NaN': NA})

In [13]:
## Convert specific columns
#numeric_columns = ['price', 'bedrooms', 'bathrooms', 'floor_area', 'land_size']
#house_dropNA[numeric_columns] = house_dropNA[numeric_columns].apply(pd.to_numeric)
#
## Convert all columns that can be converted
#house_dropNA = house_dropNA.apply(pd.to_numeric, errors='coerce')

### Price per Square Meter


To prepare the dataset for modeling, we create a Price per Square Meter variable by dividing the Price by Floor Area.

In [14]:
house_dropNA["Price per Square Meter"] = house_dropNA["price"] / house_dropNA["floor_area"]
condo_dropNA["Price per Square Meter"] = condo_dropNA["price"] / condo_dropNA["floor_area"]
apt_dropNA["Price per Square Meter"] = apt_dropNA["price"] / apt_dropNA["floor_area"]
comm_dropNA["Price per Square Meter"] = comm_dropNA["price"] / comm_dropNA["floor_area"]